# 1.Original model prediction

In [ ]:
import argparse
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from src.bio.feature_builder import CombinedPeptideFeatureBuilder
from src.bio.peptide_feature import parse_features, parse_operator
from src.config import PROJECT_ROOT
from src.data.vdjdb_source import VdjdbSource
from src.neural import evaluation
from src.processing.data_stream import DataStream
from src.processing.image_generator import ImageGenerator
from src.processing.image_padding import ImagePadding
from src.processing.padded_dataset_generator import padded_dataset_generator
from src.processing.separated_input_dataset_generator import (
    BlosumImageGenerator,
    BlosumPadding,
    separated_input_dataset_generator,
)

def fix(data_ori,data_new):
    data=pd.read_csv(data_ori)
    #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
    data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope','Affinity':'y'},inplace=True)
    df=data[['cdr3', 'antigen.epitope','y']]
    
    df.to_csv(data_new,index=False)
    
def model_predict(testfile_path,testfile_path_new,model_name,result_path, model_type="padded"):

    fix(testfile_path,testfile_path_new)
    sep_split=","#"Separator (delimiter) used for the input dataset
    cdr3_column="cdr3"
    epitope_column="antigen.epitope"
    
    
    #model_type="padded"#choices=["padded", "separated"](padded interaction map or dual input type)
    min_length_cdr3=10
    max_length_cdr3=20
    min_length_epitope=8
    max_length_epitope=11
    features="hydrophob,isoelectric,mass,hydrophil"
    operator="absdiff"#choices=["prod", "diff", "absdiff", "layer", "best"]
    batch_size=128

    # output = Path(result_path)
    # assert output.parent.exists(), "Provided output directory does not exist."

    log_fmt = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    logging.basicConfig(level=logging.INFO, format=log_fmt)
    # suppress tf logging
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # ERROR
    logging.getLogger("tensorflow").setLevel(logging.ERROR)
    logger = logging.getLogger(__name__)

    # # log arguments that were used
    # for arg, value in sorted(vars(args).items()):
    #     logging.info("CLI argument %s: %r", arg, value)

    # read in dataset
    data_source = VdjdbSource(
        filepath=testfile_path_new,
        headers={"cdr3_header": "cdr3", "epitope_header": "antigen.epitope"},
        sep=sep_split,
    )

    # filter on size
    data_source.length_filter(
        min_length_cdr3,
        max_length_cdr3,
        min_length_epitope,
        max_length_epitope,
    )

    # get list of features and operator based on input arguments
    if model_type == "padded":
        features_list = parse_features(features)
        operator = parse_operator(operator)
        feature_builder = CombinedPeptideFeatureBuilder(features_list, operator)
        logger.info("features: " + str(features_list))
        logger.info("operator: " + str(operator))
    elif model_type == "separated":
        feature_builder = None

    # store range restrictions for cdr3 and epitope
    cdr3_range = (min_length_cdr3, max_length_cdr3)
    epitope_range = (min_length_epitope, max_length_epitope)
    logger.info(f"Filtered CDR3 sequences to length: {cdr3_range}")
    logger.info(f"Filtered epitope sequences to length: {epitope_range}")

    data_stream = DataStream(
        data_source.data[
            [data_source.headers["cdr3_header"], data_source.headers["epitope_header"]]
        ].to_numpy()
    )

    # store maximum cdr3 and epitope sequence length as width and height of the 2d arrays
    cdr3_max = cdr3_range[1]
    epitope_max = epitope_range[1]

    # setup tf datasets
    if model_type == "padded":

        image_gen = ImageGenerator(data_stream, feature_builder, has_label=False)
        image_padding = ImagePadding(
            image_gen, cdr3_max, epitope_max, pad_value=0, has_label=False
        )

        padding_array = np.array([image for image in image_padding])

        dataset = tf.data.Dataset.from_tensor_slices(padding_array)
        dataset = dataset.batch(batch_size)

    elif model_type == "separated":
        # create BLOSUM encoded arrays
        blosum_encoding = BlosumImageGenerator(data_stream, has_label=False)

        # pad with zero-columns
        blosum_padding = BlosumPadding(
            blosum_encoding, cdr3_max, epitope_max, pad_value=0, has_label=False
        )

        cdr3, epitope = zip(*blosum_padding)

        dataset = (np.array(cdr3), np.array(epitope))

    # load model
    model = tf.keras.models.load_model(model_name)
    
    print('dataset:',dataset)
    # predictions
    try:
        y_pred = model.predict(dataset)
        data_source.data["y_prob"] = y_pred
    except ValueError as e:
        raise ValueError(
            "Make sure the correct model type, optional features and padding lengths are specified."
        ) from e
    data_source.data['y_pred'] =data_source.data['y_prob'].apply(lambda x: 1 if x > 0.5 else 0)
    data_source.data.rename(columns={'cdr3':'CDR3B','antigen.epitope':'Epitope','y':'y_true'},inplace=True)
    # save output
    data_source.data.to_csv(result_path+"probability.csv", index=False)


In [ ]:
testfile_path="../data/test.csv"
testfile_path_new="../data/test_new.csv"
modelfile_path="../Original_model/ImRex.h5"
result_path="../result_path/Original_model_prediction"
model_predict(testfile_path,testfile_path_new, modelfile_path,result_path)

# 2.Model retraining

In [2]:
import os
import pandas as pd
from pathlib import Path

""" Scenario for neural network with interaction map input. """
import argparse
import logging

from src.bio.feature_builder import CombinedPeptideFeatureBuilder
from src.bio.peptide_feature import parse_features, parse_operator
from src.config import PROJECT_ROOT
from src.data.control_cdr3_source import ControlCDR3Source
from src.data.vdjdb_source import VdjdbSource
from src.models.model_padded import ModelPadded
from src.scripts import io_helper
from src.neural.trainer import get_output_path, Trainer
from src.processing.cv_folds import cv_splitter
from src.processing.decoyer import Decoyer
from src.processing.inverse_map import InverseMap
from src.processing.padded_dataset_generator import padded_dataset_generator
from src.processing.splitter import splitter

def fix(data_ori,data_new):
    data=pd.read_csv(data_ori)
    #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
    data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope','Affinity':'y'},inplace=True)
    df=data[['cdr3', 'antigen.epitope','y']]
    
    df.to_csv(data_new,index=False)

def Model_retraining(trainfile_path, testfile_path, trainfile_path_new, testfile_path_new, save_model_path, result_path):

    fix(trainfile_path,trainfile_path_new)
    fix(testfile_path,testfile_path_new)
    
    run_name="try"
    depth1_1=1
    depth1_2=1
    depth2_1=1
    depth2_2=1
    
    disable_file_log= True
    reverse_augment=False
    data_path="../data/1_1_1train.csv"
    min_length_cdr3=10
    max_length_cdr3=20
    min_length_epitope=8
    max_length_epitope=15
    features="hydrophob,isoelectric,mass,hydrophil,charge"
    operator="absdiff" # choices=["prod", "diff", "absdiff", "layer", "best"]
    val_split=0.2
    cross_validation=None
    neg_augment=None
    augment_amount=None
    n_folds=None
    validation_data=None
    no_validation=True
    neg_ref=None
    full_dataset_path=None
    neg_gen_full=False
    epochs=40
    batch_size=128
    
    optimizer="rmsprop"#choices=["rmsprop", "adam", "SGD"]
    learning_rate=None
    
    gap=False
    regularization=None
    activation_function_conv="relu"
    activation_function_dense="relu"
    dropout_conv=None
    dropout_dense=None
    decoy_validation=False
    full_dataset_path=None
    epitope_ratio=False
    
    reverse_augment=False


    # create logger and log file
    run_name = io_helper.create_run_name(run_name)
    io_helper.create_logger(run_name, log_to_file=disable_file_log)
    logger = logging.getLogger(__name__)



    # read (positive) data
    data_source = VdjdbSource(
        filepath=trainfile_path_new,
        headers={"cdr3_header": "cdr3", "epitope_header": "antigen.epitope"},
    )
    # data_source.add_pos_labels()

    # filter on size
    data_source.length_filter(
        min_length_cdr3,
        max_length_cdr3,
        min_length_epitope,
        max_length_epitope,
    )
    
    # get list of features and operator based on input arguments
    features_list = parse_features(features)
    operator = parse_operator(operator)
    feature_builder = CombinedPeptideFeatureBuilder(features_list, operator)


    logger.info("features: " + str(features_list))
    logger.info("operator: " + str(operator))
    logger.info("neg_ref: " + str(neg_ref))
    logger.info("cv_type: " + str(cross_validation))
    logger.info("neg_gen_full: " + str(neg_gen_full))

    inverse_map = InverseMap()

    # store range restrictions for cdr3 and epitope
    cdr3_range = (min_length_cdr3, max_length_cdr3)
    epitope_range = (min_length_epitope, max_length_epitope)
    logger.info(f"Filtered CDR3 sequences to length: {cdr3_range}")
    logger.info(f"Filtered epitope sequences to length: {epitope_range}")

    trainer = Trainer(
        epochs,
        # include_learning_rate_reduction=args.include_learning_rate_reduction,
        # include_early_stop=args.early_stop,
        lookup=inverse_map,
        verbose=disable_file_log,
    )

    model = ModelPadded(
        width=max_length_cdr3,
        height=max_length_epitope,
        name=run_name,
        channels=feature_builder.get_number_layers(),
        optimizer=optimizer,
        learning_rate=learning_rate,
        depth1_1=depth1_1,
        depth1_2=depth1_2,
        depth2_1=depth2_1,
        depth2_2=depth2_2,
        gap=gap,
        regularization=regularization,
        activation_function_conv=activation_function_conv,
        activation_function_dense=activation_function_dense,
        dropout_conv=dropout_conv,
        dropout_dense=dropout_dense,
    )

    logger.info(f"Built model {model.base_name}:")
    # model.summary() is logged inside trainer.py




    neg_shuffle = False
    if no_validation:
        logger.info(
            "Not performing (cross-)validation; training model on all supplied data."
        )

        train = data_source

        train_output = get_output_path(
            base_name=run_name,
            file_name="train.csv",
            iteration="no_validation",
        )
        
        train_data = padded_dataset_generator(
                data_stream=train,
                feature_builder=feature_builder,
                cdr3_range=cdr3_range,
                epitope_range=epitope_range,
                neg_shuffle=neg_shuffle,
                full_dataset_path=full_dataset_path,
                epitope_ratio=epitope_ratio,
                export_path=train_output,
                neg_augment=neg_augment,
                augment_amount=augment_amount,
                reverse_augment=reverse_augment,
            )
        
        # get length of train dataset
        train_length = len(train) if not neg_shuffle else len(train) * 2

        # shuffle and batch train data
        train_data = train_data.shuffle(
            # buffer equals size of dataset, because positives and negatives are grouped
            buffer_size=train_length,
            seed=42,
            # reshuffle to make each epoch see a different order of examples
            reshuffle_each_iteration=True,
        ).batch(batch_size)

        model_instance = trainer.train(
            model=model,
            train_data=train_data,
            val_data=None,
            iteration="no_validation",
        )
        model_instance.save(save_model_path)
        # model_instance.save(
        #     filepath=get_output_path(model.base_name, model.base_name + ".h5")
        # )
        #test
        valid_source = VdjdbSource(
            filepath=testfile_path_new,
            headers={"cdr3_header": "cdr3", "epitope_header": "antigen.epitope"},
        )
        # data_source.add_pos_labels()
    
        # filter on size
        valid_source.length_filter(
            min_length_cdr3,
            max_length_cdr3,
            min_length_epitope,
            max_length_epitope,
        )
        val = valid_source
        valid_output = get_output_path(
            base_name=run_name,
            file_name="valid.csv",
            iteration="no_validation",
        )
        val_data = padded_dataset_generator(
                data_stream=val,
                feature_builder=feature_builder,
                cdr3_range=cdr3_range,
                epitope_range=epitope_range,
                inverse_map=inverse_map,
                neg_shuffle=neg_shuffle,
                full_dataset_path=full_dataset_path,
                epitope_ratio=epitope_ratio,
                export_path=valid_output,
                neg_augment=neg_augment,
                augment_amount=augment_amount,
            )
        
        # batch validation data
        val_data = val_data.batch(batch_size)
        y_pred = model_instance.predict(val_data)
        valid_source.data["y_prob"] = y_pred
        
        valid_source.data['y_pred'] =valid_source.data['y_prob'].apply(lambda x: 1 if x > 0.5 else 0)
        valid_source.data.rename(columns={'cdr3':'CDR3B','antigen.epitope':'Epitope','y':'y_true'},inplace=True)
        # save output
        valid_source.data.to_csv(result_path+"probability.csv", index=False)
        print('done saving....')
        trainer.clear_session(model_instance)

In [ ]:
trainfile_path ="../data/train.csv"
testfile_path="../data/test.csv"
trainfile_path_new="../data/train_new.csv"
testfile_path_new="../data/test_new.csv"
save_model_path="../Retraining_model/Retraining_model.h5"
result_path="../result_path/Retraining_model_prediction"
Model_retraining(trainfile_path,testfile_path, trainfile_path_new, testfile_path_new, save_model_path,result_path)

# 3. Retraining model prediction

In [7]:
import pandas as pd
from pathlib import Path
""" Scenario for neural network with interaction map input. """
import argparse
import logging

from src.bio.feature_builder import CombinedPeptideFeatureBuilder
from src.bio.peptide_feature import parse_features, parse_operator
from src.config import PROJECT_ROOT
from src.data.control_cdr3_source import ControlCDR3Source
from src.data.vdjdb_source import VdjdbSource
from src.models.model_padded import ModelPadded
from src.scripts import io_helper
from src.neural.trainer import get_output_path, Trainer
from src.processing.cv_folds import cv_splitter
from src.processing.decoyer import Decoyer
from src.processing.inverse_map import InverseMap
from src.processing.padded_dataset_generator import padded_dataset_generator
from src.processing.splitter import splitter
import tensorflow as tf

def fix(data_ori,data_new):
    data=pd.read_csv(data_ori)
    #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
    data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope','Affinity':'y'},inplace=True)
    df=data[['cdr3', 'antigen.epitope','y']]
    
    df.to_csv(data_new,index=False)


def Retraining_model_prediction(testfile_path,testfile_path_new,model_name,result_path):


    fix(testfile_path,testfile_path_new)
    
    run_name="try"
    
    min_length_cdr3=10
    max_length_cdr3=20
    min_length_epitope=8
    max_length_epitope=15
    features="hydrophob,isoelectric,mass,hydrophil,charge"
    operator="absdiff" # choices=["prod", "diff", "absdiff", "layer", "best"]
    
    neg_augment=None
    augment_amount=None
    epitope_ratio=False
    neg_ref=None
    full_dataset_path=None
    
    batch_size=128   
    
    # read (positive) data
    valid_source = VdjdbSource(
        filepath=testfile_path_new,
        headers={"cdr3_header": "cdr3", "epitope_header": "antigen.epitope"},
    )
    # data_source.add_pos_labels()

    # filter on size
    valid_source.length_filter(
        min_length_cdr3,
        max_length_cdr3,
        min_length_epitope,
        max_length_epitope,
    )

    # get list of features and operator based on input arguments
    features_list = parse_features(features)
    operator = parse_operator(operator)
    feature_builder = CombinedPeptideFeatureBuilder(features_list, operator)


    inverse_map = InverseMap()

    # store range restrictions for cdr3 and epitope
    cdr3_range = (min_length_cdr3, max_length_cdr3)
    epitope_range = (min_length_epitope, max_length_epitope)
    neg_shuffle=False
    
    val =  valid_source
    valid_output = get_output_path(
        base_name=run_name,
        file_name="valid.csv",
        iteration="no_validation",
    )
    val_data = padded_dataset_generator(
            data_stream=val,
            feature_builder=feature_builder,
            cdr3_range=cdr3_range,
            epitope_range=epitope_range,
            inverse_map=inverse_map,
            neg_shuffle=neg_shuffle,
            full_dataset_path=full_dataset_path,
            epitope_ratio=epitope_ratio,
            export_path=valid_output,
            neg_augment=neg_augment,
            augment_amount=augment_amount,
        )
    
    # batch validation data
    val_data = val_data.batch(batch_size)
    # y_pred = model_instance.predict(val_data)
    
    # load model
    model = tf.keras.models.load_model(model_name)

    # predictions
    try:
        y_pred = model.predict(val_data)
        valid_source.data["y_prob"] = y_pred
    except ValueError as e:
        raise ValueError(
            "Make sure the correct model type, optional features and padding lengths are specified."
        ) from e
    valid_source.data['y_pred'] =valid_source.data['y_prob'].apply(lambda x: 1 if x > 0.5 else 0)
    valid_source.data.rename(columns={'cdr3':'CDR3B','antigen.epitope':'Epitope','y':'y_true'},inplace=True)
    # save output
    valid_source.data.to_csv(result_path+"probability.csv", index=False)
    print('done saving....')

In [ ]:
testfile_path="../data/Validation.csv"
testfile_path_new="../data/Validation_new.csv"
modelfile_path="../Retraining_model/Retraining_model.h5"
result_path="../result_path/Retraining_model_prediction"
Retraining_model_prediction(testfile_path,testfile_path_new,modelfile_path,result_path)